## Reto 5: Automatizando peticiones

### 1. Objetivos:
    - Automatizar la petición de datos a la API de la NASA por fechas.
 
---
    
### 2. Desarrollo:

### a) Automatizando peticiones

En el primer Reto realizaste una petición a la API de la NASA para pedir 5 datos de la hoja #100. Ahora, vamos a automatizar el proceso de realizar peticiones a la API para obtener una cantidad bastante mayor de datos.

Vamos a obtener ahora los datos por fechas, así que tendrás que ir a la página de la [api de la NASA](https://api.nasa.gov/), para revisar cómo es posible realizar peticiones por fecha. Queremos obtener los **dos primeros meses (Enero y Febrero) del año 1995**. Observa que la API sólo permite peticiones por fecha en rangos de 7 días.

Tu reto tiene los siguientes pasos:

1. Revisa la documentación de la API de la NASA para entender cómo realizar peticiones por fecha.
2. Realiza una primera petición de prueba para entender el formato de los datos que obtienes de regreso (cómo extraemos los datos que necesitamos y qué estructura tienen).
2. Escribe el código necesario para automatizar las peticiones a la API y obtener los meses de Enero y Febrero del año 1995.
3. Almacena los datos de cada petición y luego usa esos datos para crear `DataFrames`.
4. Concatena verticalmente tus `DataFrames` para obtener un `DataFrame` final que contenga todos los datos de tus peticiones. Cada fila tiene que corresponder a un objeto espacial.
5. Guarda tu `DataFrame` con el nombre de `near_earth_objects-january_february_1995-raw.csv`.

> **Nota**: En este momento no te preocupes por explorar o limpiar tu dataset. Eso lo haremos en la siguiente sesión. Lo que sí tienes que asegurarte es de normalizar tus datos antes de convertirlos en `DataFrame`.

¡Mucha suerte!

Importamos las librerias necesarias

In [97]:
import pandas as pd 
import requests
from pprint import pprint

Definimos el endpoint e ingresamos los parametros 

In [113]:
payload = {'api_key':'QAXLpeREJyy8YUciiVBq0v42tuc0Hjn6Izb6DiZj','start_date':'1995-01-01','end_date':'1995-01-01'}
endpoint = "https://api.nasa.gov/neo/rest/v1/feed"

Realizamos una primera petición de prueba

In [114]:
r = requests.get(endpoint,payload)
r.status_code

200

In [117]:
json = r.json()

data = json['near_earth_objects']

normalized = pd.json_normalize(data,record_path=['1995-01-01'])

objetos = pd.DataFrame.from_dict(normalized)

objetos.head()

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,3703080,3703080,(2014 YW34),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3703080,21.70,True,"[{'close_approach_date': '1995-01-01', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3703080?ap...,0.121494,0.271669,121.494041,271.668934,0.075493,0.168807,398.602509,891.302306
1,3723955,3723955,(2015 NK3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3723955,21.30,False,"[{'close_approach_date': '1995-01-01', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3723955?ap...,0.146068,0.326618,146.067964,326.617897,0.090762,0.202951,479.225620,1071.581063
2,3792437,3792437,(2017 XR61),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3792437,23.10,False,"[{'close_approach_date': '1995-01-01', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3792437?ap...,0.063761,0.142574,63.760979,142.573883,0.039619,0.088591,209.189570,467.762099
3,54105518,54105518,(2021 AW5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54105518,26.54,False,"[{'close_approach_date': '1995-01-01', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/54105518?a...,0.013078,0.029244,13.078411,29.244216,0.008127,0.018172,42.908173,95.945593


In [163]:
payload = {'api_key':'QAXLpeREJyy8YUciiVBq0v42tuc0Hjn6Izb6DiZj','start_date':'1995-01-01','end_date':'1995-01-01'}
endpoint = "https://api.nasa.gov/neo/rest/v1/feed"

dict_datos = {}
for i in range(0,59):
  try:
    r = requests.get(endpoint, params=payload)

    if r.status_code == 200:
      json = r.json()

      data = json['near_earth_objects']
      dict_datos[i] = data

      new_link = json['links']['next']
      endpoint = new_link
    
  except:
    continue

In [164]:
len(dict_datos)

59

In [165]:
lista_fechas = pd.date_range(start="1995-01-01",end="1995-02-28").strftime('%Y-%m-%d')

len(lista_fechas)

59

In [166]:
for par in dict_datos:
  normalizados = pd.json_normalize(dict_datos[par],record_path=lista_fechas[par])
  df = pd.DataFrame.from_dict(normalizados)
  dict_datos[par] = df

In [167]:
lista_de_dataframes = []

for key in dict_datos:
    lista_de_dataframes.append(dict_datos[key])

In [168]:
df_completo = pd.concat(lista_de_dataframes, axis=0).reset_index(drop=True)

In [169]:
df_completo

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,3703080,3703080,(2014 YW34),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3703080,21.70,True,"[{'close_approach_date': '1995-01-01', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3703080?ap...,0.121494,0.271669,121.494041,271.668934,0.075493,0.168807,398.602509,891.302306
1,3723955,3723955,(2015 NK3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3723955,21.30,False,"[{'close_approach_date': '1995-01-01', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3723955?ap...,0.146068,0.326618,146.067964,326.617897,0.090762,0.202951,479.225620,1071.581063
2,3792437,3792437,(2017 XR61),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3792437,23.10,False,"[{'close_approach_date': '1995-01-01', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3792437?ap...,0.063761,0.142574,63.760979,142.573883,0.039619,0.088591,209.189570,467.762099
3,54105518,54105518,(2021 AW5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54105518,26.54,False,"[{'close_approach_date': '1995-01-01', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/54105518?a...,0.013078,0.029244,13.078411,29.244216,0.008127,0.018172,42.908173,95.945593
4,3553279,3553279,(2010 XB24),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3553279,21.80,True,"[{'close_approach_date': '1995-01-02', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3553279?ap...,0.116026,0.259442,116.025908,259.441818,0.072095,0.161210,380.662441,851.187094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,3763475,3763475,(2016 VL3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3763475,24.30,False,"[{'close_approach_date': '1995-02-28', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3763475?ap...,0.036691,0.082043,36.690614,82.042706,0.022798,0.050979,120.376033,269.168993
391,3781651,3781651,(2017 SS10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3781651,25.20,False,"[{'close_approach_date': '1995-02-28', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3781651?ap...,0.024241,0.054205,24.241248,54.205079,0.015063,0.033681,79.531656,177.838190
392,3781897,3781897,(2017 SL16),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3781897,25.90,False,"[{'close_approach_date': '1995-02-28', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3781897?ap...,0.017561,0.039268,17.561232,39.268108,0.010912,0.024400,57.615592,128.832380
393,54097866,54097866,(2020 XF),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54097866,26.84,False,"[{'close_approach_date': '1995-02-28', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/54097866?a...,0.011391,0.025471,11.390820,25.470647,0.007078,0.015827,37.371457,83.565118


In [170]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [172]:
#df_completo.to_csv('../../Datasets/near_earth_objects-january_february_1995-raw.csv')